<a href="https://colab.research.google.com/github/dalatorrem/Diplomado_Metodos_UCentral_2023/blob/main/Lectura_28_practica_analisis_lineal_mod_class_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo de predicción del puntaje en la prueba saber 11

Contexto Analítico: <br>
* construir un modelo lineal que teniendo como variables de entrada variables sociodemográficas nos permita dar una predicción del puntaje global.
* Construir un modelo de clasificación que permita identificar la probabilidad de que una persona que presenta la prueba tenga un puntaje global por debajo de 200.

In [51]:
import pandas                   as pd
import numpy                    as np
import matplotlib.pyplot        as plt
import seaborn                  as sns
from   scipy                    import stats
import statsmodels.formula.api  as smf
import scipy
import warnings
from sklearn.model_selection    import train_test_split
from sklearn.linear_model       import LinearRegression
from sklearn.metrics            import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics            import mean_absolute_percentage_error, roc_auc_score
import plotly.express as px

In [5]:
path_data = 'https://raw.githubusercontent.com/dalatorrem/Diplomado_Metodos_UCentral_2023/main/data/Resultados__nicos_Saber_11_sample.csv'
data_saber = pd.read_csv(path_data)
data_saber

,Unnamed: 0,PERIODO,ESTU_TIPODOCUMENTO,ESTU_CONSECUTIVO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,...,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,DESEMP_INGLES,PUNT_INGLES,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_LECTURA_CRITICA,PUNT_GLOBAL
0,0,20131,CR,SB11201310000414,URBANO,N,B,ACADÉMICO,3.118480e+11,3.118480e+11,...,Si,Si,Si,B+,94.0,88.0,NaN,NaN,NaN,NaN
1,1,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
2,2,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
3,3,20122,TI,SB11201220204399,URBANO,N,A,TÉCNICO/ACADÉMICO,1.631300e+11,1.631300e+11,...,Si,Si,No,A1,48.0,45.0,NaN,NaN,NaN,NaN
4,4,20132,TI,SB11201320464198,URBANO,N,A,TÉCNICO,1.190010e+11,1.190010e+11,...,Si,Si,Si,A-,43.0,52.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,20162,TI,SB11201620287491,RURAL,NaN,A,ACADÉMICO,2.687200e+11,2.687200e+11,...,No,No,No,A1,55.0,50.0,57.0,48.0,53.0,262.0
9996,9996,20132,TI,SB11201320276289,URBANO,N,A,TÉCNICO/ACADÉMICO,2.470010e+11,2.470010e+11,...,Si,Si,No,A-,39.0,57.0,NaN,NaN,NaN,NaN
9997,9997,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0
9998,9998,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0


In [14]:
list_vars_select = ['ESTU_TIPODOCUMENTO',  'COLE_BILINGUE',
                    'COLE_CALENDARIO',  'COLE_GENERO',  'COLE_JORNADA',
                    'ESTU_DEPTO_PRESENTACION',  'ESTU_ESTADOINVESTIGACION',
                    'ESTU_GENERO',  'FAMI_EDUCACIONMADRE',  'FAMI_EDUCACIONPADRE',
                    'FAMI_ESTRATOVIVIENDA',  'FAMI_TIENEAUTOMOVIL',
                    'FAMI_TIENECOMPUTADOR',  'ESTU_FECHANACIMIENTO', 'PERIODO', 'PUNT_GLOBAL']
r_state = 20230830
TARGET = 'PUNT_GLOBAL'

In [7]:
data_modelo_con_edad = data_saber[list_vars_select].dropna().copy()
df_train, df_test = train_test_split(data_modelo_con_edad, test_size=0.25, random_state=r_state)
len(df_train), len(df_test), len(df_train)/len(data_modelo_con_edad), len(df_test)/len(data_modelo_con_edad)

(3498, 1166, 0.75, 0.25)

In [8]:
df_train['AÑO_NACIMIENTO'] = [int(x[-4:]) if int(x[-4:])>100 else int(x[-4:])+2000 for x in df_train['ESTU_FECHANACIMIENTO']]
df_train['AÑO_PRESENTACION'] = [int(str(x)[:4]) for x in df_train['PERIODO']]
df_train['EDAD'] = df_train['AÑO_PRESENTACION'] - df_train['AÑO_NACIMIENTO']
df_train[['AÑO_NACIMIENTO', 'ESTU_FECHANACIMIENTO', 'AÑO_PRESENTACION', 'PERIODO', 'EDAD']].sample(10)

,AÑO_NACIMIENTO,ESTU_FECHANACIMIENTO,AÑO_PRESENTACION,PERIODO,EDAD
4752,1998,24/03/1998,2017,20172,19
7415,2002,20/10/2002,2019,20194,17
8308,1996,17/08/1996,2014,20142,18
9153,2001,23/05/2001,2017,20172,16
1592,1997,15/08/1997,2015,20152,18
2683,1998,31/05/1998,2016,20162,18
2650,2001,24/08/2001,2019,20191,18
8909,1999,18/02/1999,2015,20152,16
8831,1992,03/12/1992,2016,20162,24
4378,2001,20/02/2001,2017,20172,16


In [9]:
df_test['AÑO_NACIMIENTO'] = [int(x[-4:]) if int(x[-4:])>100 else int(x[-4:])+2000 for x in df_test['ESTU_FECHANACIMIENTO']]
df_test['AÑO_PRESENTACION'] = [int(str(x)[:4]) for x in df_test['PERIODO']]
df_test['EDAD'] = df_test['AÑO_PRESENTACION'] - df_test['AÑO_NACIMIENTO']
df_test[['AÑO_NACIMIENTO', 'ESTU_FECHANACIMIENTO', 'AÑO_PRESENTACION', 'PERIODO', 'EDAD']].sample(10)

,AÑO_NACIMIENTO,ESTU_FECHANACIMIENTO,AÑO_PRESENTACION,PERIODO,EDAD
7122,1999,10/03/1999,2015,20152,16
7524,1997,14/08/1997,2014,20142,17
3127,2002,20/05/2002,2019,20194,17
7678,1997,29/11/1997,2014,20142,17
1425,2002,19/09/2002,2019,20194,17
3953,2001,23/02/2001,2019,20194,18
7840,2001,20/07/2001,2019,20194,18
3663,2000,21/04/2000,2017,20172,17
5703,1998,26/09/1998,2016,20162,18
9123,2001,03/12/2001,2019,20194,18


In [10]:
df_train = df_train[(df_train['EDAD']>10)].copy()
df_train['EDAD'].describe()

count    3493.000000
mean       17.760092
std         3.050311
min        15.000000
25%        17.000000
50%        17.000000
75%        18.000000
max        59.000000
Name: EDAD, dtype: float64

# Preprocesamiento

In [12]:
vars_categoricas = ['ESTU_TIPODOCUMENTO',  'COLE_BILINGUE',
                    'COLE_CALENDARIO',  'COLE_GENERO',  'COLE_JORNADA',
                    'ESTU_DEPTO_PRESENTACION',  'ESTU_ESTADOINVESTIGACION',
                    'ESTU_GENERO',  'FAMI_EDUCACIONMADRE',  'FAMI_EDUCACIONPADRE',
                    'FAMI_ESTRATOVIVIENDA',  'FAMI_TIENEAUTOMOVIL',
                    'FAMI_TIENECOMPUTADOR']

In [13]:
for col in vars_categoricas:
  list_rep = list(set(df_train[col].unique()) & set(df_test[col].unique()))
  val_counts = df_train[col].value_counts(1)
  val_counts = val_counts[val_counts>0.05]  # si está por debajo del 5% iría a otro
  list_rep = [x for x in list_rep if x in val_counts.index]
  df_train[col] = [x if x in list_rep else 'otro' for x in df_train[col]]
  df_test[col] = [x if x in list_rep else 'otro' for x in df_test[col]]

# Model Sin transformación

In [15]:
formula_3 = f'{TARGET} ~ '\
 + ' + '.join([x for x in vars_categoricas]) + ' + EDAD'
formula_3

'PUNT_GLOBAL ~ ESTU_TIPODOCUMENTO + COLE_BILINGUE + COLE_CALENDARIO + COLE_GENERO + COLE_JORNADA + ESTU_DEPTO_PRESENTACION + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + EDAD'

In [16]:
model_3 = smf.ols(formula=formula_3, data=df_train).fit()
r_2_summary = round(model_3.rsquared, 3)
r_2_train_cal = round(r2_score(df_train[TARGET], model_3.predict(df_train)),3)

r_2_test_cal = round(r2_score(df_test[TARGET], model_3.predict(df_test)),3)

print(f'Modelo tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo tiene un R2 Resumen_modelo, train, test (0.306, 0.306, 0.26)


In [17]:
model_3.resid

8633    10.172847
1628   -50.481549
8204   -19.857409
7734    -9.074158
4027   -45.914167
          ...    
8308   -52.844374
5772    56.364250
5933     1.960401
7987     4.141347
9580    -1.720805
Length: 3493, dtype: float64

In [20]:
model_3.predict(df_train)

8633    329.827153
1628    308.481549
8204    226.857409
7734    287.074158
4027    231.914167
           ...    
8308    255.844374
5772    256.635750
5933    256.039599
7987    278.858653
9580    253.720805
Length: 3493, dtype: float64

In [33]:
df_train_con_resid = df_train.copy()
df_train_con_resid['Predicciones'] = model_3.predict(df_train_con_resid)
df_train_con_resid['RESIDUALES_CAL'] = df_train_con_resid[TARGET] - df_train_con_resid['Predicciones']
df_train_con_resid['RESIDUALES_MODEL'] = model_3.resid
df_train_con_resid['DIF_RESID'] = df_train_con_resid['RESIDUALES_CAL'] - df_train_con_resid['RESIDUALES_MODEL']
df_train_con_resid['DIF_RESID'].describe()

count    3493.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: DIF_RESID, dtype: float64

In [34]:
df_train_con_resid

,ESTU_TIPODOCUMENTO,COLE_BILINGUE,COLE_CALENDARIO,COLE_GENERO,COLE_JORNADA,ESTU_DEPTO_PRESENTACION,ESTU_ESTADOINVESTIGACION,ESTU_GENERO,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,...,ESTU_FECHANACIMIENTO,PERIODO,PUNT_GLOBAL,AÑO_NACIMIENTO,AÑO_PRESENTACION,EDAD,Predicciones,RESIDUALES_CAL,RESIDUALES_MODEL,DIF_RESID
8633,CC,otro,otro,MIXTO,COMPLETA,BOGOTÁ,PUBLICAR,F,otro,otro,...,01/01/2000,20181,340.0,2000,2018,18,329.827153,10.172847,10.172847,0.0
1628,TI,N,A,MIXTO,COMPLETA,BOGOTÁ,PUBLICAR,M,otro,otro,...,30/10/1997,20152,258.0,1997,2015,18,308.481549,-50.481549,-50.481549,0.0
8204,TI,N,A,MIXTO,TARDE,otro,PUBLICAR,F,Primaria completa,Primaria incompleta,...,11/12/2001,20194,207.0,2001,2019,18,226.857409,-19.857409,-19.857409,0.0
7734,TI,N,A,MIXTO,COMPLETA,BOGOTÁ,PUBLICAR,F,Secundaria (Bachillerato) completa,otro,...,01/12/2001,20194,278.0,2001,2019,18,287.074158,-9.074158,-9.074158,0.0
4027,TI,N,A,MIXTO,TARDE,ATLANTICO,PUBLICAR,F,Secundaria (Bachillerato) completa,otro,...,30/04/1997,20152,186.0,1997,2015,18,231.914167,-45.914167,-45.914167,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8308,TI,N,A,MIXTO,TARDE,ANTIOQUIA,PUBLICAR,F,Educación profesional completa,Primaria incompleta,...,17/08/1996,20142,203.0,1996,2014,18,255.844374,-52.844374,-52.844374,0.0
5772,TI,N,A,MIXTO,otro,BOGOTÁ,PUBLICAR,F,otro,Secundaria (Bachillerato) incompleta,...,31/03/2002,20194,313.0,2002,2019,17,256.635750,56.364250,56.364250,0.0
5933,TI,N,A,MIXTO,MAÑANA,CUNDINAMARCA,PUBLICAR,M,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,...,19/07/1999,20162,258.0,1999,2016,17,256.039599,1.960401,1.960401,0.0
7987,TI,N,A,MIXTO,MAÑANA,otro,PUBLICAR,M,Secundaria (Bachillerato) incompleta,Educación profesional completa,...,22/04/2000,20162,283.0,2000,2016,16,278.858653,4.141347,4.141347,0.0


In [35]:
px.histogram(df_train_con_resid['RESIDUALES_CAL'])

In [36]:
px.scatter(df_train_con_resid, x=f'EDAD', y='RESIDUALES_CAL')

In [24]:
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     42.29
Date:                Thu, 31 Aug 2023   Prob (F-statistic):          7.64e-243
Time:                        23:42:22   Log-Likelihood:                -17934.
No. Observations:                3493   AIC:                         3.594e+04
Df Residuals:                    3456   BIC:                         3.617e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

# Model Log - Log

In [25]:
formula_4 = f'np.log({TARGET}) ~ '\
 + ' + '.join([x for x in vars_categoricas]) + ' + np.log(EDAD)'
formula_4

'np.log(PUNT_GLOBAL) ~ ESTU_TIPODOCUMENTO + COLE_BILINGUE + COLE_CALENDARIO + COLE_GENERO + COLE_JORNADA + ESTU_DEPTO_PRESENTACION + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + np.log(EDAD)'

In [27]:
model_4 = smf.ols(formula=formula_4, data=df_train).fit()
r_2_summary = round(model_4.rsquared, 3)
r_2_train_cal = round(r2_score(np.log(df_train[TARGET]), model_4.predict(df_train)),3)

r_2_test_cal = round(r2_score(np.log(df_test[TARGET]), model_4.predict(df_test)),3)

print(f'Modelo tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo tiene un R2 Resumen_modelo, train, test (0.292, 0.292, 0.258)


$Y = X\beta +\epsilon$

$\epsilon \~ N(0, \sigma^2)$


In [28]:
model_4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     np.log(PUNT_GLOBAL)   R-squared:                       0.292
Model:                             OLS   Adj. R-squared:                  0.285
Method:                  Least Squares   F-statistic:                     39.62
Date:                 Thu, 31 Aug 2023   Prob (F-statistic):          1.37e-228
Time:                         23:47:36   Log-Likelihood:                 1372.1
No. Observations:                 3493   AIC:                            -2670.
Df Residuals:                     3456   BIC:                            -2442.
Df Model:                           36                                         
Covariance Type:             nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       5.9165      0.090     65.633      0.000       5.740       6.093
ESTU_TIPODOCUMENTO[T.TI]                                        0.0285      0.010      2.963      0.003       0.010       0.047
ESTU_TIPODOCUMENTO[T.otro]                                      0.0063      0.025      0.256      0.798      -0.042       0.055
COLE_BILINGUE[T.otro]                                           0.0484      0.019      2.539      0.011       0.011       0.086
COLE_CALENDARIO[T.otro]                                         0.0783      0.016      4.819      0.000       0.046       0.110
COLE_GENERO[T.otro]                                             0.0614      0.014      4.317      0.000       0.034       0.089
COLE_JORNADA[T.MAÑANA]                                         -0.0515      0.008     -6.372      0.000      -0.067      -0.036
COLE_JORNADA[T.NOCHE]                                          -0.1581      0.014    -11.569      0.000      -0.185      -0.131
COLE_JORNADA[T.SABATINA]                                       -0.1322      0.015     -8.684      0.000      -0.162      -0.102
COLE_JORNADA[T.TARDE]                                          -0.0497      0.011     -4.720      0.000      -0.070      -0.029
COLE_JORNADA[T.otro]                                           -0.0554      0.015     -3.739      0.000      -0.084      -0.026
ESTU_DEPTO_PRESENTACION[T.ATLANTICO]                           -0.0234      0.014     -1.684      0.092      -0.051       0.004
ESTU_DEPTO_PRESENTACION[T.BOGOTÁ]                               0.0302      0.010      2.890      0.004       0.010       0.051
ESTU_DEPTO_PRESENTACION[T.CUNDINAMARCA]                         0.0212      0.014      1.540      0.124      -0.006       0.048
ESTU_DEPTO_PRESENTACION[T.SANTANDER]                            0.0722      0.015      4.932      0.000       0.044       0.101
ESTU_DEPTO_PRESENTACION[T.VALLE]                               -0.0115      0.013     -0.918      0.359      -0.036       0.013
ESTU_DEPTO_PRESENTACION[T.otro]                                 0.0040      0.009      0.430      0.667      -0.014       0.022
ESTU_ESTADOINVESTIGACION[T.otro]                               -0.2187      0.165     -1.326      0.185      -0.542       0.105
ESTU_GENERO[T.M]                                                0.0275      0.006      4.839      0.000       0.016       0.039
FAMI_EDUCACIONMADRE[T.Primaria completa]                       -0.0491      0.014     -3.614      0.000      -0.076      -0.022
FAMI_EDUCACIONMADRE[T.Primaria incompleta]                     -0.0461      0.014     -3.338      0.001      -0.073      -0.019


In [29]:
df_train_con_resid = df_train.copy()
df_train_con_resid['Predicciones'] = model_4.predict(df_train_con_resid)
df_train_con_resid['RESIDUALES_CAL'] = np.log(df_train_con_resid[TARGET]) - df_train_con_resid['Predicciones']
df_train_con_resid['RESIDUALES_MODEL'] = model_4.resid
df_train_con_resid['DIF_RESID'] = df_train_con_resid['RESIDUALES_CAL'] - df_train_con_resid['RESIDUALES_MODEL']
df_train_con_resid['DIF_RESID'].describe()

count    3493.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: DIF_RESID, dtype: float64

** Observe que la apliacación de la trnsformació log - log generó que se creará una cola a la izquierda, la asimetría está más alejada de 0.**

In [30]:
px.histogram(df_train_con_resid['RESIDUALES_CAL'])

In [32]:
df_train_con_resid[f'LOG_EDAD'] = np.log(df_train_con_resid['EDAD'])
px.scatter(df_train_con_resid, x=f'LOG_EDAD', y='RESIDUALES_CAL')

# Ejemplo Regresión logística

Construir un modelo de clasificación que permita identificar la probabilidad de que una persona que presenta la prueba tenga un puntaje global por debajo de 200.

In [37]:
df_train_cla = df_train.copy()
df_test_cla = df_test.copy()

* 1 Corresponde a estar por debajo de 200
* 0 Ser mayor o igua a 200

In [38]:
TARGET

'PUNT_GLOBAL'

In [39]:
TARGET_CLA = 'PUNT_MENOR_200'
df_train_cla[TARGET_CLA] =  1*(df_train_cla[TARGET]<200)
df_test_cla[TARGET_CLA] =  1*(df_test_cla[TARGET]<200)

In [40]:
df_train_cla[TARGET_CLA].value_counts()

0    3034
1     459
Name: PUNT_MENOR_200, dtype: int64

In [41]:
df_train_cla[TARGET_CLA].value_counts(1)

0    0.868594
1    0.131406
Name: PUNT_MENOR_200, dtype: float64

In [42]:
df_test_cla[TARGET_CLA].value_counts()

0    1016
1     150
Name: PUNT_MENOR_200, dtype: int64

In [43]:
df_test_cla[TARGET_CLA].value_counts(1)

0    0.871355
1    0.128645
Name: PUNT_MENOR_200, dtype: float64

## Balanceo de clases

Subsampling

In [46]:
(df_train_cla[TARGET_CLA]==1).sum()

459

In [47]:
df_balanceado = df_train_cla[(df_train_cla[TARGET_CLA]==0)]\
                  .sample((df_train_cla[TARGET_CLA]==1).sum()).copy()
df_balanceado = pd.concat([df_balanceado, df_train_cla[(df_train_cla[TARGET_CLA]==1)]])
df_balanceado[TARGET_CLA].value_counts()

0    459
1    459
Name: PUNT_MENOR_200, dtype: int64

***PARA DESBALANCEO PUEDE CONSULTAR:***

https://imbalanced-learn.org/stable/auto_examples/combine/plot_comparison_combine.html#sphx-glr-auto-examples-combine-plot-comparison-combine-py

## Modelo logístico

In [48]:
formula_logit = f'{TARGET_CLA} ~ '\
 + ' + '.join([x for x in vars_categoricas]) + ' + EDAD'
formula_logit

'PUNT_MENOR_200 ~ ESTU_TIPODOCUMENTO + COLE_BILINGUE + COLE_CALENDARIO + COLE_GENERO + COLE_JORNADA + ESTU_DEPTO_PRESENTACION + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + EDAD'

In [50]:
lr_logit = smf.logit(formula=formula_logit, data=df_train_cla).fit()
print(lr_logit.summary())

         Current function value: 0.340582
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:         PUNT_MENOR_200   No. Observations:                 3493
Model:                          Logit   Df Residuals:                     3456
Method:                           MLE   Df Model:                           36
Date:                Fri, 01 Sep 2023   Pseudo R-squ.:                  0.1246
Time:                        00:27:49   Log-Likelihood:                -1189.7
converged:                      False   LL-Null:                       -1359.0
Covariance Type:            nonrobust   LLR p-value:                 7.176e-51
                                                                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                     

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [53]:
lr_logit.predict(df_train_cla)

8633    0.062846
1628    0.047300
8204    0.228028
7734    0.034343
4027    0.240307
          ...   
8308    0.123573
5772    0.115678
5933    0.067402
7987    0.046517
9580    0.069029
Length: 3493, dtype: float64

In [54]:
roc_auc_score(df_train_cla[TARGET_CLA], lr_logit.predict(df_train_cla))

0.7499852793970441

In [55]:
roc_auc_score(df_test_cla[TARGET_CLA], lr_logit.predict(df_test_cla))

0.7283727034120735

In [56]:
df_test_cla['PREDICCIONES'] = lr_logit.predict(df_test_cla)

In [57]:
px.box(df_test_cla, orientation='h', x='PREDICCIONES', y=TARGET_CLA)

Suponga que se debe implementar una medida sobre aquellas personas que van a presentar la prueba y tienen una probabilidad considerable de obtener un puntaje por debaje de 200. ¿Qué umbral de probabilidad colocaría para realizar la intervención?

In [67]:
umbral_intervencion = 0.35

df_test_cla['INTERVENCION'] = ['SI' if x>=umbral_intervencion else 'NO' for x in df_test_cla['PREDICCIONES']]
pd.crosstab(df_test_cla[TARGET_CLA], df_test_cla['INTERVENCION'])

INTERVENCION,NO,SI
PUNT_MENOR_200,,
0,987,29
1,111,39


In [69]:
umbral_intervencion = 0.5

df_test_cla['INTERVENCION'] = ['SI' if x>=umbral_intervencion else 'NO' for x in df_test_cla['PREDICCIONES']]
pd.crosstab(df_test_cla[TARGET_CLA], df_test_cla['INTERVENCION'])

INTERVENCION,NO,SI
PUNT_MENOR_200,,
0,1009,7
1,137,13


In [70]:
len(df_test_cla)

1166

In [72]:
df_test_cla.sort_values('PREDICCIONES',ascending=False).head(66)

,ESTU_TIPODOCUMENTO,COLE_BILINGUE,COLE_CALENDARIO,COLE_GENERO,COLE_JORNADA,ESTU_DEPTO_PRESENTACION,ESTU_ESTADOINVESTIGACION,ESTU_GENERO,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,...,FAMI_TIENECOMPUTADOR,ESTU_FECHANACIMIENTO,PERIODO,PUNT_GLOBAL,AÑO_NACIMIENTO,AÑO_PRESENTACION,EDAD,PUNT_MENOR_200,PREDICCIONES,INTERVENCION
8194,CC,N,A,MIXTO,NOCHE,ATLANTICO,PUBLICAR,F,Primaria incompleta,Primaria incompleta,...,No,08/04/1979,20162,154.0,1979,2016,37,1,0.767078,SI
9705,CC,N,A,MIXTO,NOCHE,ATLANTICO,PUBLICAR,F,Primaria incompleta,Primaria incompleta,...,No,13/01/1986,20194,261.0,1986,2019,33,0,0.748492,SI
596,CC,N,A,MIXTO,NOCHE,VALLE,PUBLICAR,F,Primaria incompleta,Primaria incompleta,...,No,23/05/1980,20162,177.0,1980,2016,36,1,0.692746,SI
2924,CC,N,A,MIXTO,NOCHE,otro,PUBLICAR,F,Primaria incompleta,Primaria incompleta,...,No,18/09/1979,20152,131.0,1979,2015,36,1,0.689967,SI
4786,CC,N,A,MIXTO,NOCHE,otro,PUBLICAR,M,Primaria incompleta,Primaria incompleta,...,No,11/12/1982,20194,160.0,1982,2019,37,1,0.654901,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,CC,N,A,MIXTO,SABATINA,otro,PUBLICAR,M,otro,otro,...,No,14/02/1979,20162,201.0,1979,2016,37,0,0.360663,NO
433,CC,N,A,MIXTO,NOCHE,otro,PUBLICAR,M,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,...,No,19/03/1998,20162,193.0,1998,2016,18,1,0.360609,NO
7941,TI,N,A,MIXTO,NOCHE,ATLANTICO,PUBLICAR,F,Educación profesional completa,Técnica o tecnológica completa,...,No,28/01/2002,20194,315.0,2002,2019,17,0,0.357656,NO
8721,CC,N,A,MIXTO,SABATINA,otro,PUBLICAR,F,Primaria completa,otro,...,No,12/12/1993,20152,185.0,1993,2015,22,1,0.357459,NO


In [74]:
umbral_intervencion = 0.357177

df_test_cla['INTERVENCION'] = ['SI' if x>=umbral_intervencion else 'NO' for x in df_test_cla['PREDICCIONES']]
pd.crosstab(df_test_cla[TARGET_CLA], df_test_cla['INTERVENCION'])

INTERVENCION,NO,SI
PUNT_MENOR_200,,
0,988,28
1,112,38


In [76]:
38/150

0.25333333333333335

In [77]:
28/(988+ 28)

0.027559055118110236